<a href="https://www.kaggle.com/code/aayushsin7a/service-capacity-estimation-analysis?scriptVersionId=179319312" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=e61820cbadd7ecdfb8dd8d39bf80a606ded50a123bf16fc7182b9e262a661bce
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import sys
print(sys.version)


3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]


In [3]:
import pyspark
print(pyspark.__version__)


3.5.1


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from pyspark.sql.functions import *
import os
import sys
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
#sc=pyspark.SparkContext()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
#import sys
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import lead, lag, col ,datediff, to_date, lit, size , length , year, month, dayofmonth, from_unixtime, unix_timestamp, desc, asc, rank, min , expr, trim, date_format, avg
import datetime
from datetime import datetime
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType, TimestampType, StructField, NumericType
from pyspark.sql.functions import isnan
from pyspark.sql.functions import udf
from io import BytesIO
#import boto3
import math
from pyspark.sql import Row
#import pyarrow as pa
import pandas as pd
from datetime import datetime, timedelta
#from time import gmtime, strftime ,datetime
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import count, min,max
from pyspark.sql.functions import col, when
#import geohash2 as pgh
#import pygeohash as pgh2
spark = SparkSession.builder.appName('NE').getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 09:49:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[key: string, value: string]

In [5]:
df_tele = spark.read.parquet("/kaggle/input/april-data/April.parquet")

In [6]:
df_tele.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- vehicleId: string (nullable = true)
 |-- gpsLatitude: double (nullable = true)
 |-- gpsLongitude: double (nullable = true)
 |-- odometer_count: long (nullable = true)
 |-- District: string (nullable = true)
 |-- eventdate: timestamp_ntz (nullable = true)



In [7]:
#df_tele.select('vehicleId').distinct().count()

1)Read telemetery
2) vincount/total vincount based on gpslat,gpslong,1month
3) District mapping
4) Mode calculation -> if necessary 
4)region/sso/workshop data read 
5)Distance calculation
7)s3 to save the file ( later append all 6 months ) 

In [8]:
df_tele = df_tele.drop("Unnamed: 0")

In [9]:
df_tele.show(5,False)


+-----------------+------------------+-----------------+--------------+------------+-------------------+
|vehicleId        |gpsLatitude       |gpsLongitude     |odometer_count|District    |eventdate          |
+-----------------+------------------+-----------------+--------------+------------+-------------------+
|MAT373328J1P37808|18.90111329999999 |73.01143329999996|0             |Raigarh     |2023-04-01 00:00:00|
|MAT373370G2F14206|17.88427489999998 |74.80197829999997|0             |Solapur     |2023-04-01 00:00:00|
|MAT373370G2G16127|17.469103199999978|78.42705660000001|0             |Hydrabad    |2023-04-01 00:00:00|
|MAT373370G2G16947|16.7756282        |78.13296489999999|0             |Mahabubnagar|2023-04-01 00:00:00|
|MAT373370H2A00579|17.037436599999978|79.415955        |0             |Nalgonda    |2023-04-01 00:00:00|
+-----------------+------------------+-----------------+--------------+------------+-------------------+
only showing top 5 rows



# District Mapping

In [ ]:
#districts = gpd.read_file("/kaggle/input/district-shapefiles/output.shp")

In [ ]:
'''
# District Mapping Function 
def get_district(long, lat):
    point = Point(long, lat)
    for index, row in districts.iterrows():
        if row['geometry'].contains(point):
            return row['distname']
    return 'District not found'
'''

In [10]:
# Register the UDF
#get_district_udf = udf(get_district, StringType())

In [12]:
#df_tele = df_tele.withColumn('District', get_district_udf(df_tele['gpsLongitude'].cast('float'), df_tele['gpsLatitude'].cast('float')))
df_tele = df_tele.withColumn("eventdate", to_date(df_tele["eventdate"], "yyyy-MM-dd"))
df_tele = df_tele.withColumn('coordinates', concat(df_tele['gpsLatitude'], lit(','), df_tele['gpsLongitude']))
df_tele = df_tele.withColumn("eventdate", to_date(df_tele["eventdate"], "yyyy-MM-dd"))
# Extract the month number from the event date
df_tele = df_tele.withColumn("month", month(df_tele["eventdate"]))

In [ ]:
#df_tele.show()

In [13]:
df_region = spark.read.csv("/kaggle/input/workshops/all_region_workshops.csv",header=True)

In [14]:
df_region.show(3)
print(df_region.count())

+--------------------+---------+----------+------+----------+
|            Division|Lattitude| Longitude|   SSO|     State|
+--------------------+---------+----------+------+----------+
|100B520-Sv&Pa-Nag...|8.1908371|77.4192042| Salem|Tamil Nadu|
|1001650-Sv&Pa-Nag...| 8.198264| 77.431443| Salem|Tamil Nadu|
|2088645-Sv&Pa-Ney...|   8.3681|  77.08208|Cochin|    Kerala|
+--------------------+---------+----------+------+----------+
only showing top 3 rows

1592


# Python 

In [ ]:
'''
import pandas as pd
from openpyxl import load_workbook
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm
from math import ceil
import numpy as np
import folium
import warnings
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.cluster import KMeans
from statistics import mode
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN
from haversine import haversine
from haversine import Unit

warnings.filterwarnings('ignore')
geolocator = Nominatim(user_agent="geoapiExercises")
'''


In [ ]:
'''

def cal_mode(data, latitude, longitude):
    Returns : the first value if all the values are different


    # Combining to find mode for Lat & Long both
    data['combined'] = data[latitude] + ' - ' + data[longitude]

    # Calculating mode
    mod = data.groupby(['vehicleId', 'weekofMonth', 'District'])['combined'].apply(lambda x: mode(x)).reset_index() 

    for index, val in enumerate(mod['combined']):
        mod.loc[index, 'Lat'] = val.split(' -')[0]
        mod.loc[index, 'Long'] = val.split('- ')[1]

    mod.drop('combined', axis=1, inplace=True)

    return mod

# def haversine(lonlat1, lonlat2):
#     """
#     Calculate the great circle distance between two points
#     on the earth (specified in decimal degrees)
#     """
#     # convert decimal degrees to radians
#     lat1, lon1 = lonlat1
#     lat2, lon2 = lonlat2
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
#
#     # haversine formula
#     dlon = lon2 - lon1
#     dlat = lat2 - lat1
#     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
#     c = 2 * asin(sqrt(a))
#     r = 6371  # Radius of earth in kilometers. Use 3956 for miles
#     return c * r

def week_of_month(dt):
    """ Returns the week of the month for the specified date."""
    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom / 7.0))
    
'''

In [ ]:
#df_tele.count()

In [ ]:
#df_filtered_region = df_region.filter(col("SSO").isin("Delhi", "Lucknow"))

In [ ]:
df_tele.printSchema()

In [ ]:
#df_tele = df_tele.withColumn("eventdate",to_date(col("eventdate"),"yyyy-MM-dd"))

In [ ]:
#df_tele.show(4)

In [ ]:
'''

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, dayofmonth, floor
from pyspark.sql.types import IntegerType
from math import ceil

# Initialize a Spark session
#spark = SparkSession.builder.appName("WeekOfMonth").getOrCreate()

# Assuming you have a DataFrame named df_tele
# Convert the "eventdate" column to date type
df_tele = df_tele.withColumn("eventdate", to_date(df_tele["eventdate"], "yyyy-MM-dd"))

# Calculate the day of the month
df_tele = df_tele.withColumn("day_of_month", dayofmonth(df_tele["eventdate"]))

# Define a UDF to calculate the week of the month
from pyspark.sql.functions import udf

def week_of_month(dt):
    """ Returns the week of the month for the specified date."""
    dom = dt.day
    adjusted_dom = dom + (dt.weekday() + 1)  # Adjusted for 1-based indexing
    return int(ceil(adjusted_dom / 7.0))

# Register the UDF
week_udf = udf(week_of_month, IntegerType())

# Calculate the week of the month
df_tele = df_tele.withColumn("week_of_month", week_udf(df_tele["eventdate"]))

# Show the resulting DataFrame
df_tele.show(3)

'''

In [ ]:
#df_tele = df_tele.withColumn("eventdate", to_date(df_tele["eventdate"], "yyyy-MM-dd"))

# Extract the month number from the event date
#df_tele = df_tele.withColumn("month", month(df_tele["eventdate"]))

from pyspark.sql.functions import year, month, weekofyear, dayofmonth, dayofweek, dayofyear, hour, minute, second, unix_timestamp
from pyspark.sql import DataFrame

def add_datepart_spark(df: DataFrame, field_name: str, prefix=None, drop=True, time=False) -> DataFrame:
    # Extract the date-related features using PySpark functions
    df = df.withColumn("Year", year(field_name))
    df = df.withColumn("Month", month(field_name))
    df = df.withColumn("Week", weekofyear(field_name))
    df = df.withColumn("Day", dayofmonth(field_name))
    df = df.withColumn("Dayofweek", dayofweek(field_name))
    df = df.withColumn("Dayofyear", dayofyear(field_name))
    df = df.withColumn("Is_month_end", (dayofmonth(field_name) == 31).cast("int"))
    df = df.withColumn("Is_month_start", (dayofmonth(field_name) == 1).cast("int"))
    df = df.withColumn("Is_quarter_end", (month(field_name) == 3).cast("int"))
    df = df.withColumn("Is_quarter_start", (month(field_name) == 1).cast("int"))
    df = df.withColumn("Is_year_end", (month(field_name) == 12).cast("int"))
    df = df.withColumn("Is_year_start", (month(field_name) == 1).cast("int"))

    #if time:
        #df = df.withColumn("Hour", hour(field_name))
        #df = df.withColumn("Minute", minute(field_name))
        #df = df.withColumn("Second", second(field_name))

    # Add the "Elapsed" column
    #df = df.withColumn("Elapsed", (unix_timestamp(field_name) * 1000).cast("long"))

    # Drop the original date column if drop is True
    #if drop:
        #df = df.drop(field_name)

    return df

# Usage:
df_tele = add_datepart_spark(df_tele, df_tele["eventdate"], prefix='event')


In [ ]:
#df_tele.printSchema()

In [ ]:
#df_tele.select("eventdate","day_of_month","week_of_month","Dayofweek","Dayofyear").show(10)

In [ ]:
'''
# WEEK_OF_MONTH
import pyspark.sql.functions as F

def cal_mode(data, latitude, longitude):
    # Combine latitude and longitude into a single column
    data = data.withColumn('combined', F.concat_ws(' - ', data[latitude], data[longitude]))

    # Calculate mode using a custom function
    mode_udf = F.udf(lambda x: x.mode()[0] if len(x) > 0 else None)
    mod = data.groupby(['vehicleId', 'week_of_month', 'District']).agg(mode_udf(F.collect_list('combined')).alias('mode'))

    # Split the combined column into 'Lat' and 'Long'
    mod = mod.withColumn('Lat', F.split(mod['mode'], ' - ')[0])
    mod = mod.withColumn('Long', F.split(mod['mode'], ' - ')[1])

    # Drop the intermediate 'combined' column
    mod = mod.drop('mode')

    return mod

# This is where you define the df_mode variable
df_mode = cal_mode(df_tele, 'gpsLatitude', 'gpsLongitude')
'''

In [ ]:
#print(df_mode.count())
#df_mode.show(3)

# WEEK OF MONTH

In [ ]:
# WEEK OF MONTH
df_tele = df_tele.withColumn(
    "week_of_month",
    expr("ceil(dayofmonth(eventdate)/7)")
)

In [15]:
import pyspark.sql.functions as F

def cal_mode_month(data, latitude, longitude):
    # Combining latitude and longitude into a single column
    data = data.withColumn('combined', F.concat_ws(' - ', data[latitude], data[longitude]))
    # Calculating mode using PySpark's aggregation functions
    mod = data.groupby(['vehicleId', 'month', 'District']) \
              .agg(F.expr('collect_list(combined) as combined_list')) \
              .withColumn('mode', F.expr('aggregate(combined_list, cast(null as string), (acc, x) -> IFNULL(acc, x), acc -> acc)'))
    # Splitting the combined column into 'Lat' and 'Long'
    mod = mod.withColumn('Lat', F.split(mod['mode'], ' - ')[0])
    mod = mod.withColumn('Long', F.split(mod['mode'], ' - ')[1])
    # Drop the intermediate columns
    mod = mod.drop('combined', 'combined_list', 'mode')
    return mod

# This is where you define the df_mode variable
df_mode1 = cal_mode_month(df_tele, 'gpsLatitude', 'gpsLongitude')


In [ ]:
#df_mode1.count()

In [16]:
df_mode1 = df_mode1.withColumn('Lat', col('Lat').cast("float"))
# Cast 'Long' column to Float
df_mode1 = df_mode1.withColumn('Long', col('Long').cast("float"))
# Cast 'District' column to String (assuming it's not already a string)
df_mode1 = df_mode1.withColumn('District', col('District').cast("string"))

In [17]:
df_mode1.columns

['vehicleId', 'month', 'District', 'Lat', 'Long']

In [18]:
from pyspark.sql.functions import round
# Assuming 'df_mode' is your PySpark DataFrame
# Round 'Lat' column to two decimal places
df_mode1 = df_mode1.withColumn('Lat', round(df_mode1['Lat'], 2))
# Round 'Long' column to two decimal places
df_mode1 = df_mode1.withColumn('Long', round(df_mode1['Long'], 2))


In [19]:

from pyspark.sql import functions as F

# Group by 'District' and count unique combinations of 'Lat' and 'Long'
rou_off = df_mode1.groupBy('District', 'Lat', 'Long').agg(F.count("*").alias("count"))



# Region 

In [20]:
df_region.columns

['Division', 'Lattitude', 'Longitude', 'SSO', 'State']

# SSO

In [21]:
state_sso_dis = spark.read.csv("/kaggle/input/workshops/District to SSO master.csv",header=True,inferSchema=True)

In [22]:
state_sso_dis.show(4)

+--------------------+---------+--------------+----------+
|            District|vehicleId|         State|       SSO|
+--------------------+---------+--------------+----------+
|            Adilabad|        3|     Telangana| Hyderabad|
|                Agra|       85| Uttar Pradesh|   Lucknow|
|         Kadapa(YSR)|       40|Andhra Pradesh| Hyderabad|
|Sri Potti Sriramu...|       32|Andhra Pradesh|Vijayawada|
+--------------------+---------+--------------+----------+
only showing top 4 rows



# Join - sso with round off

In [23]:
#df_mode1 = rou_off.join(df_region_renamed, on='District', how='inner')
df_model = rou_off.join(state_sso_dis[['District','SSO','State']], on='District', how='inner')
#print(df_mode1.count())

In [25]:
df_model.select('SSO').distinct().show()

+-----------+
|        SSO|
+-----------+
|  Bangalore|
|    Udaipur|
|Bhubaneswar|
|      Jammu|
|      Hubli|
|      Patna|
|    Lucknow|
|    Chennai|
|     Mumbai|
|  Ahmedabad|
|    Kolkata|
|  Allahabad|
|      Salem|
|   Lucknow |
|   Dehradun|
|    Gurgaon|
|       Pune|
|      Delhi|
|        JSR|
|       #N/A|
+-----------+
only showing top 20 rows



In [ ]:
'''
import pyspark.sql.functions as F

def filter_df(df, sso_col, sso_list):
    '''
    Filters the DataFrame to only the rows where the sso column is in the sso_list
    '''

    # Creating a filter expression
    filter_expr = F.col(sso_col).isin(sso_list)

    # Filtering the DataFrame
    df_filtered = df.filter(filter_expr)

    return df_filtered
'''

In [26]:
#For all SSO
# Assuming df_model is your DataFrame
#unique_sso_values = df_model.select('SSO').distinct().rdd.map(lambda row: row[0]).collect()

# Filtering the DataFrame
#del_cha = filter_df(df_model, 'SSO', unique_sso_values)

# Assuming df_model is already defined
unique_sso_df = df_model.select('SSO').distinct()

# Broadcast join for efficiency
del_cha = df_model.join(broadcast(unique_sso_df), on='SSO', how='inner')


In [27]:
## delhi
#work_reg = filter_df(df_region,'SSO',['Delhi', 'Lucknow'])
#unique_sso_values_reg = df_region.select('SSO').distinct().rdd.map(lambda row: row[0]).collect()
#work_reg = filter_df(df_region, 'SSO',unique_sso_values_reg) 

from pyspark.sql.functions import broadcast

# Assuming df_region is already defined
unique_sso_df_reg = df_region.select('SSO').distinct()
# Broadcast join for efficiency
work_reg = df_region.join(broadcast(unique_sso_df_reg), on='SSO', how='inner')



In [ ]:
#work_reg.show(4)

In [ ]:
#work_reg.count()
# delhi / lucknow - count 92 

In [28]:
del_cha.columns

['SSO', 'District', 'Lat', 'Long', 'count', 'State']

In [29]:
work_reg.columns

['SSO', 'Division', 'Lattitude', 'Longitude', 'State']

In [30]:
work_reg = work_reg.withColumnRenamed("Lattitude", "Latitude")


In [31]:
work_reg.columns

['SSO', 'Division', 'Latitude', 'Longitude', 'State']

In [32]:
del_cha.printSchema()

root
 |-- SSO: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Lat: float (nullable = true)
 |-- Long: float (nullable = true)
 |-- count: long (nullable = false)
 |-- State: string (nullable = true)



In [ ]:
#del_cha = del_cha.withColumnRenamed("State", "state")

In [33]:
work_reg.printSchema()

root
 |-- SSO: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- State: string (nullable = true)



In [ ]:
'''

from pyspark.sql.functions import col

# Convert Latitude and Longitude columns to double type
work_reg = work_reg.withColumn("Latitude", col("Latitude").cast("double"))
work_reg = work_reg.withColumn("Longitude", col("Longitude").cast("double"))

# Verify the updated schema
work_reg.printSchema()

'''


In [ ]:
'''

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculates the Haversine distance between two points in latitude and longitude.
    
    Args:
        lat1 (float): The latitude of the first point.
        lon1 (float): The longitude of the first point.
        lat2 (float): The latitude of the second point.
        lon2 (float): The longitude of the second point.
    
    Returns:
        float: The Haversine distance between the two points in kilometers.
    """
    R = 6371.00885
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    a = math.sin(dLat / 2) ** 2 + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    return R * c
'''

In [ ]:
'''


# Register the UDF
haversine_udf = udf(haversine, DoubleType())

# Cross join the DataFrames to create all combinations
result_df = del_cha.crossJoin(work_reg)

# Calculate distances using the UDF and add them as a new column
# Calculate distances using the UDF and add them as a new column
result_df = result_df.withColumn("Distance", haversine_udf(
    del_cha["Latitude"], del_cha["Longitude"],
    work_reg["Latitude"], work_reg["Longitude"]
))

# Use window functions to find the minimum distance for each 'District'
window_spec = Window.partitionBy(result_df["District"]).orderBy(result_df["Distance"])
result_df = result_df.withColumn("MinDistance", spark_min(result_df["Distance"]).over(window_spec))

# Select the relevant columns (e.g., 'District', 'Division', and 'MinDistance')
result_df = result_df.select("District", "Division", "MinDistance").distinct()

# Show the resulting DataFrame
result_df.show()
'''

In [ ]:
'''


from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

# Define Haversine function
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    import math

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    # Radius of the Earth in kilometers
    R = 6371
    distance = R * c

    return distance

# Register Haversine function as a UDF
haversine_udf = F.udf(haversine, DoubleType())

# Cross join the DataFrames to create all combinations
result_df = del_cha.crossJoin(work_reg)

# Calculate distances using the UDF and add them as a new column
result_df = result_df.withColumn("Distance", haversine_udf(
    del_cha["Latitude"], del_cha["Longitude"],
    work_reg["Latitude"], work_reg["Longitude"]
))

# Use window functions to find the minimum distance for each 'District'
window_spec = Window.partitionBy(result_df["District"]).orderBy(result_df["Distance"])
result_df = result_df.withColumn("MinDistance", F.min(result_df["Distance"]).over(window_spec))

# Select the relevant columns (e.g., 'District', 'Division', and 'MinDistance')
result_df = result_df.select(del_cha["District"], work_reg["SSO"], del_cha["Latitude"].alias("Del_Latitude"), del_cha["Longitude"].alias("Del_Longitude"), del_cha["count"], del_cha["State"], (F.col("MinDistance") * F.lit(1000)).alias("MinDistance")).distinct()
result_df = result_df.orderBy("MinDistance")
# Show the resulting DataFrame
result_df.show()
'''

In [ ]:
#result_df.show(5)

In [ ]:
## test

In [ ]:
'''
def haversine(lat1, lon1, lat2, lon2):
    import math
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Radius of the Earth in kilometers
    R = 6371
    distance = R * c
    
    return distance
'''

In [ ]:
''' 
#Register Haversine function as a UDF
haversine_udf = F.udf(haversine, DoubleType())

# Cross join the DataFrames to create all combinations
result_df = del_cha.crossJoin(work_reg)

# Calculate distances using the UDF and add them as a new column
result_df = result_df.withColumn("Distance", haversine_udf(
    del_cha["Latitude"], del_cha["Longitude"],
    work_reg["Latitude"], work_reg["Longitude"]
))

# Use window functions to find the minimum distance for each 'District'
window_spec = Window.partitionBy(result_df["District"]).orderBy(result_df["Distance"])
result_df = result_df.withColumn("MinDistance", F.min(result_df["Distance"]).over(window_spec))

# Select the relevant columns (e.g., 'District', 'Division', and 'MinDistance')
result_df = result_df.select(del_cha["District"], work_reg["SSO"], del_cha["Latitude"].alias("Del_Latitude"), del_cha["Longitude"].alias("Del_Longitude"), del_cha["count"], del_cha["State"], (F.col("MinDistance") * F.lit(1000)).alias("MinDistance")).distinct()
result_df = result_df.orderBy("MinDistance")
# Show the resulting DataFrame
result_df.show()

'''

In [ ]:
#result_df_filtered = result_df.filter(result_df["State"] == "Uttar Pradesh")

# Show the resulting filtered DataFrame
#result_df_filtered.show()

In [ ]:
#result_df_filtered.count()

In [ ]:
rs = del_cha
station = work_reg

In [ ]:
rs.printSchema()

In [ ]:
rs = rs.withColumn("Lat", col("Lat").cast("double"))
rs = rs.withColumn("Long", col("Long").cast("double"))

In [ ]:
station.printSchema()

In [ ]:
station = station.withColumnRenamed("Latitude", "sLat")
station = station.withColumnRenamed("Longitude", "sLong")

In [ ]:
station = station.withColumn("sLat", col("sLat").cast("double"))
station = station.withColumn("sLong", col("sLong").cast("double"))

In [ ]:
station.printSchema()

In [34]:
#from pyspark.sql.functions import F, lit

# Haversine UDF (User-Defined Function)
def haversine_udf(lon1, lat1, lon2, lat2):
  """
  UDF to calculate the great-circle distance between two points on a sphere using the Haversine formula.

  Args:
      lon1 (float): Longitude of the first point in degrees.
      lat1 (float): Latitude of the first point in degrees.
      lon2 (float): Longitude of the second point in degrees.
      lat2 (float): Latitude of the second point in degrees.

  Returns:
      float: The distance between the two points in kilometers.
  """
  # Convert decimal degrees to radians
  lon1_rad = lon1 * F.lit(math.pi / 180.0)
  lat1_rad = lat1 * F.lit(math.pi / 180.0)
  lon2_rad = lon2 * F.lit(math.pi / 180.0)
  lat2_rad = lat2 * F.lit(math.pi / 180.0)

  # Haversine formula calculations
  dlon = lon2_rad - lon1_rad
  dlat = lat2_rad - lat1_rad
  a = F.sin(dlat / 2) ** 2 + F.cos(lat1_rad) * F.cos(lat2_rad) * F.sin(dlon / 2) ** 2
  c = 2 * F.asin(F.sqrt(a))
  # Radius of Earth in kilometers (change to miles if needed)
  R = lit(6371.0)

  return c * R


In [ ]:
'''
rs_with_distance = rs.withColumn(
    "distance_to_station",
    haversine_udf(
        rs["Long"],
        rs["Lat"],
        F.lit(station.select("sLong").head()[0]),  # Use F.lit() to convert Python value to column
        F.lit(station.select("sLat").head()[0])   # Use F.lit() to convert Python value to column
    )
)

# Print the DataFrame with distance
#rs_with_distance.show()

'''

In [ ]:
rs_with_distance = rs_with_distance.orderBy("distance_to_station")


In [ ]:
#rs_with_distance.show()

In [ ]:
#print(result_df_filtered.count())
#print(rs_with_distance.count())

In [37]:
del_cha.columns

['SSO', 'District', 'Lat', 'Long', 'count', 'State']

In [39]:
work_reg.columns

['SSO', 'Division', 'Latitude', 'Longitude', 'State']

In [41]:


del_cha_distance = del_cha.withColumn(
    "Min_distance",
    haversine_udf(
        del_cha["Lat"],
        del_cha["Long"],
        F.lit(work_reg.select("Longitude").head()[0]),  # Use F.lit() to convert Python value to column
        F.lit(work_reg.select("Latitude").head()[0])   # Use F.lit() to convert Python value to column
    )
)


In [42]:
del_cha_distance.show()

+----------+---------------+-----+-----+-----+-------------+------------------+
|       SSO|       District|  Lat| Long|count|        State|      Min_distance|
+----------+---------------+-----+-----+-----+-------------+------------------+
|   Lucknow|          Unnao|26.62| 80.7|  465|Uttar Pradesh| 8452.296540655547|
| Bangalore|         Mandya|12.55|76.88|    3|    Karnataka| 8501.674200129422|
|     Patna|      Begusarai| 25.5|85.94|   21|        Bihar| 8819.902824728599|
|    Jaipur|         Jaipur|26.82| 75.5|   22|    Rajasthan| 8098.125789513314|
|   Udaipur|        Jodhpur|27.16|73.15|    3|    Rajasthan| 7933.799897772404|
|Chandigarh|           Moga|30.83|75.08|    4|       Punjab| 7980.661725422702|
|     Patna|Kaimur (Bhabua)|25.22|83.51|   56|        Bihar| 8658.759288416753|
|     Delhi|     South West|28.57|77.06|   54| NCT of Delhi| 8168.241279177264|
|   Kolkata|          Haora|22.59|88.22|  559|  West Bengal| 8983.063668175757|
|     Patna|     Kishanganj|26.37|88.07|

In [ ]:
#del_cha_distance.toPandas().to_csv("/kaggle/working/output.csv")

In [ ]:
#rs_with_distance.head(10)

In [56]:
del_cha = del_cha.filter(col("Long").isNotNull() & col("Lat").isNotNull())
work_reg = work_reg.filter(col("Longitude").isNotNull() & col("Latitude").isNotNull())


In [57]:
import math
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType

def haversine(lon1, lat1, lon2, lat2):
    if None in [lon1, lat1, lon2, lat2]:
        return None
    lon1_rad = lon1 * (math.pi / 180.0)
    lat1_rad = lat1 * (math.pi / 180.0)
    lon2_rad = lon2 * (math.pi / 180.0)
    lat2_rad = lat2 * (math.pi / 180.0)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = (math.sin(dlat / 2) ** 2) + math.cos(lat1_rad) * math.cos(lat2_rad) * (math.sin(dlon / 2) ** 2)
    c = 2 * math.asin(math.sqrt(a))
    R = 6371.0  # Radius of Earth in kilometers
    return c * R

haversine_udf = udf(haversine, DoubleType())


In [58]:
haversine_udf = udf(haversine, DoubleType())

In [59]:
broadcast_station = broadcast(work_reg)

In [60]:
del_cha_distance = del_cha.crossJoin(broadcast_station) \
    .withColumn("distance_to_station", haversine_udf(col("Long"), col("Lat"), col("Longitude"), col("Latitude")))

In [61]:
del_cha_distance = del_cha_distance.orderBy("distance_to_station")

In [62]:
del_cha_distance.show()

24/05/23 10:27:48 ERROR Executor: Exception in task 1.0 in stage 133.0 (TID 167)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_33/4181433418.py", line 10, in haversine
TypeError: can't multiply sequence by non-int of type 'float'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_33/4181433418.py", line 10, in haversine
TypeError: can't multiply sequence by non-int of type 'float'


In [63]:
import math
from pyspark.sql.functions import udf, col, broadcast
from pyspark.sql.types import DoubleType

def haversine(lon1, lat1, lon2, lat2):
    try:
        lon1 = float(lon1)
        lat1 = float(lat1)
        lon2 = float(lon2)
        lat2 = float(lat2)
    except (TypeError, ValueError):
        return None

    if None in [lon1, lat1, lon2, lat2]:
        return None

    lon1_rad = lon1 * (math.pi / 180.0)
    lat1_rad = lat1 * (math.pi / 180.0)
    lon2_rad = lon2 * (math.pi / 180.0)
    lat2_rad = lat2 * (math.pi / 180.0)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = (math.sin(dlat / 2) ** 2) + math.cos(lat1_rad) * math.cos(lat2_rad) * (math.sin(dlon / 2) ** 2)
    c = 2 * math.asin(math.sqrt(a))
    R = 6371.0  # Radius of Earth in kilometers
    return c * R


In [64]:
haversine_udf = udf(haversine, DoubleType())

In [65]:
# Filter out rows with None values for longitude and latitude
del_cha = del_cha.filter(col("Long").isNotNull() & col("Lat").isNotNull())
work_reg = work_reg.filter(col("Longitude").isNotNull() & col("Latitude").isNotNull())


In [66]:
# Explicitly cast the columns to float
del_cha = del_cha.withColumn("Long", col("Long").cast("float"))
del_cha = del_cha.withColumn("Lat", col("Lat").cast("float"))
work_reg = work_reg.withColumn("Longitude", col("Longitude").cast("float"))
work_reg = work_reg.withColumn("Latitude", col("Latitude").cast("float"))


In [67]:
# Broadcast the station DataFrame
broadcast_station = broadcast(work_reg)

In [68]:
# Perform join and calculate distances
del_cha_distance = del_cha.crossJoin(broadcast_station) \
    .withColumn("distance_to_station", haversine_udf(col("Long"), col("Lat"), col("Longitude"), col("Latitude")))


In [71]:

# Order by distance
del_cha_distance = del_cha_distance.orderBy("distance_to_station")

In [ ]:
del_cha_distance.show(10,False)